In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.optimize import linprog
import joblib
import os

# Initialize Flask app
app = Flask(__name__)

# Load trained XGBoost model
model_path = "./xgb_model.pkl"
xgb_model = joblib.load(model_path)

# Function to preprocess input features
def preprocess_input(data):
    """
    Convert natural language input or CSV data into structured model features.
    """
    # Ensure necessary columns exist
    required_columns = ['latency_ms', 'packet_loss_rate_percent', 'load_percent', 'active_users',
                        'spectrum_mhz', 'handover_attempts', 'handover_failures', 'rsrp_dBm',
                        'rsrq_dB', 'sinr_dB', 'spectrum_utilization_percent', 'congestion_level_percent']
    
    for col in required_columns:
        if col not in data:
            data[col] = 0  # Default missing values to 0
    
    return data

# Function for LP-based resource allocation
def allocate_resources(df):
    df['total_capacity'] = 1 * (1 - df["spectrum_utilization_percent"]) * (1 - df["load_percent"])
    df["available_capacity"] = df["total_capacity"] * (1 - df["spectrum_utilization_percent"]) * (1 - df["load_percent"])
    
    # LP Optimization Setup
    c = -df["model_prediction"].values  # Objective: Maximize allocation
    num_cells = len(df)
    A_ub = np.eye(num_cells)
    b_ub = np.minimum(df["available_capacity"].values, df['total_capacity'] * 0.7)  # 70% max allocation
    
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, method='highs')
    df["allocated_resources"] = result.x if result.success else np.zeros(num_cells)
    return df

# API Endpoint: Predict single query
@app.route('/predict_single', methods=['POST'])
def predict_single():
    try:
        data = request.json
        data = preprocess_input(data)
        df = pd.DataFrame([data])
        prediction = xgb_model.predict(df)[0]
        df["model_prediction"] = prediction
        df = allocate_resources(df)
        return jsonify({"predicted_traffic_mbps": prediction, "allocated_resources_mbps": df["allocated_resources"].values[0]})
    except Exception as e:
        return jsonify({"error": str(e)})

# API Endpoint: Batch CSV processing
@app.route('/predict_batch', methods=['POST'])
def predict_batch():
    try:
        file = request.files['file']
        df = pd.read_csv(file)
        df = preprocess_input(df)
        df['model_prediction'] = xgb_model.predict(df)
        df = allocate_resources(df)
        output_path = "./output/predictions.csv"
        os.makedirs("./output", exist_ok=True)
        df.to_csv(output_path, index=False)
        return jsonify({"message": "Batch processing complete", "download_link": output_path})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.100:5000
Press CTRL+C to quit
 * Restarting with stat
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/novachrono/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/novachrono/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1052, in launch_instance
    app.initialize(argv)
  File "/Users/novachrono/Library/Python/3.11/lib/python/site-packages/traitlets/config/a

SystemExit: 1

/Users/novachrono/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.optimize import linprog
import joblib
import os

# Initialize Flask app
app = Flask(__name__)

# Root endpoint to check if API is running
@app.route('/')
def home():
    return "Flask API is running!"

# Load trained XGBoost model
model_path = "./xgb_model.pkl"
xgb_model = joblib.load(model_path)

# Function to preprocess input features
def preprocess_input(data):
    """
    Convert natural language input or CSV data into structured model features.
    """
    required_columns = [
        'Unnamed: 0', 'latency_ms', 'packet_loss_rate_percent', 'load_percent', 'active_users',
        'spectrum_mhz', 'handover_attempts', 'handover_failures', 'rsrp_dBm', 'rsrq_dB',
        'sinr_dB', 'spectrum_utilization_percent', 'congestion_level_percent', 'year_x', 'month_x',
        'week_x', 'daily_x', 'hour_x', 'dayofweek_x', 'lag_1_x', 'lag_3_x', 'sin_day_of_week_x',
        'cos_day_of_week_x', 'band_Band1', 'band_Band2', 'band_Band3', 'issue_description_Equipment failure',
        'issue_description_Network instability', 'issue_description_Overload', 'issue_description_Signal loss',
        'year_y', 'month_y', 'week_y', 'daily_y', 'hour_y', 'dayofweek_y', 'lag_1_y', 'lag_3_y',
        'sin_day_of_week_y', 'cos_day_of_week_y'
    ]
    df = pd.DataFrame([data]) if isinstance(data, dict) else data
    
    for col in required_columns:
        if col not in df.columns:
            df[col] = 0  # Fill missing columns with default value
    
    return df[required_columns]  # Ensure correct column order

# Function for LP-based resource allocation
def allocate_resources(df):
    df['total_capacity'] = 1 * (1 - df["spectrum_utilization_percent"]) * (1 - df["load_percent"])
    df["available_capacity"] = df["total_capacity"] * (1 - df["spectrum_utilization_percent"]) * (1 - df["load_percent"])
    
    # LP Optimization Setup
    c = -df["model_prediction"].values  # Objective: Maximize allocation
    num_cells = len(df)
    A_ub = np.eye(num_cells)
    b_ub = np.minimum(df["available_capacity"].values, df['total_capacity'] * 0.7)  # 70% max allocation
    
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, method='highs')
    df["allocated_resources"] = result.x if result.success else np.zeros(num_cells)
    return df

# API Endpoint: Predict single query
@app.route('/predict_single', methods=['POST'])
def predict_single():
    try:
        data = request.json
        df = preprocess_input(data)
        prediction = float(xgb_model.predict(df)[0])  # Convert float32 to standard float
        df["model_prediction"] = prediction
        df = allocate_resources(df)
        allocated_resources = float(df["allocated_resources"].values[0])  # Convert to standard float
        return jsonify({"predicted_traffic_mbps": prediction, "allocated_resources_mbps": allocated_resources})
    except Exception as e:
        return jsonify({"error": str(e)})


# API Endpoint: Batch CSV processing
@app.route('/predict_batch', methods=['POST'])
def predict_batch():
    try:
        file = request.files['file']
        df = pd.read_csv(file)
        df = preprocess_input(df)
        df['model_prediction'] = xgb_model.predict(df)
        df = allocate_resources(df)
        output_path = "./output/predictions.csv"
        os.makedirs("./output", exist_ok=True)
        df.to_csv(output_path, index=False)
        return jsonify({"message": "Batch processing complete", "download_link": output_path})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.100:5000
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2025 18:13:40] "POST /predict_single HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2025 18:13:49] "GET / HTTP/1.1" 200 -


In [7]:
pip install transformers torch


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from pandasai.llm.huggingface import HuggingFace

ModuleNotFoundError: No module named 'pandasai.llm.huggingface'

In [2]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import os
from pandasai import SmartDataframe
from pandasai.llm.huggingface import HuggingFace  # Use Hugging Face model

# Initialize Flask app
app = Flask(__name__, template_folder="templates")

# Load CSV file at startup
data_path = "./test_set_feature_prediction_2.csv"  # Ensure this file exists
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
else:
    df = pd.DataFrame()

# Initialize PandasAI for AI-powered CSV querying
llm = HuggingFace(model_name="tiiuae/falcon-7b-instruct")  # Replace with a suitable free model
df_ai = SmartDataframe(df, config={"llm": llm})

@app.route('/')
def home():
    return render_template("index.html")

# API Endpoint: Query CSV with AI based on cell ID
@app.route('/query_csv', methods=['POST'])
def query_csv():
    try:
        query = request.json.get("query", "")
        if not query:
            return jsonify({"error": "No query provided"})
        
        result = df_ai.chat(query)
        return jsonify({"response": str(result)})
    except Exception as e:
        return jsonify({"error": str(e)})

# API Endpoint: Chatbot for cell-based queries
@app.route('/chatbot', methods=['POST'])
def chatbot():
    try:
        user_query = request.json.get("query", "").lower()
        if not user_query:
            return jsonify({"error": "No query provided"})
        
        # Extract cell ID from user query
        words = user_query.split()
        cell_id = next((word for word in words if word.isdigit()), None)
        
        if cell_id is None:
            return jsonify({"error": "No valid cell ID found in query"})
        
        # Search for the cell ID in the CSV
        cell_data = df[df['cellid'].astype(str) == cell_id]
        if cell_data.empty:
            return jsonify({"error": f"Cell ID {cell_id} not found in data"})
        
        # Extract network usage and allocated resources
        predicted_traffic = cell_data['predicted_traffic_mbps'].values[0]
        allocated_resources = cell_data['allocated_resources_mbps'].values[0]
        
        response = {
            "query": user_query,
            "cell_id": cell_id,
            "predicted_traffic_mbps": predicted_traffic,
            "allocated_resources_mbps": allocated_resources
        }
        return jsonify(response)
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)


ModuleNotFoundError: No module named 'pandasai.llm.huggingface'

In [2]:
pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 13.4 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 18.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, jsonify, render_template
import pandas as pd
import os

# Initialize Flask app
app = Flask(__name__, template_folder="templates")

# Load CSV file
data_path = "./ResourceAllocation.csv"
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
else:
    df = pd.DataFrame()

@app.route('/')
def home():
    return render_template("index.html")

# API Endpoint: Query CSV by cell_id
@app.route('/query_csv', methods=['POST'])
def query_csv():
    try:
        cell_id = request.json.get("cell_id", "").lower()
        if not cell_id:
            return jsonify({"error": "No cell_id provided"})

        if df.empty:
            return jsonify({"error": "CSV file is empty or missing"})

        df["cell_id"] = df["cell_id"].astype(str).str.lower()
        filtered_df = df[df["cell_id"] == cell_id]
        
        if filtered_df.empty:
            return jsonify({"error": "No matching cell_id found"})
        
        latest_entry = filtered_df.sort_values(by="timestamp", ascending=False).iloc[0]
        model_prediction = latest_entry["model_prediction"]
        capacity_allocated = latest_entry["Capacity_Allocated(Mbps)"]
        
        response_text = (f"For cell {cell_id}, the predicted network traffic is {model_prediction}, "
                         f"and the needed resource for this traffic is {capacity_allocated} Mbps.")
        
        return jsonify({"response": response_text})
    except Exception as e:b
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.100:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Mar/2025 10:54:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 10:54:45] "POST /query_csv HTTP/1.1" 200 -


In [ ]:
from flask import Flask, request, jsonify, render_template, send_file
import pandas as pd
import os

# Initialize Flask app
app = Flask(__name__, template_folder="templates", static_folder="static")

# Load CSV file
data_path = "./ResourceAllocation.csv"
if os.path.exists(data_path):
    df = pd.read_csv(data_path)
else:
    df = pd.DataFrame()

@app.route('/')
def home():
    return render_template("index.html")

# API Endpoint: Query CSV by cell_id
@app.route('/query_csv', methods=['POST'])
def query_csv():
    try:
        cell_id = request.json.get("cell_id", "").strip().lower()
        if not cell_id:
            return jsonify({"error": "No cell_id provided"})

        if df.empty:
            return jsonify({"error": "CSV file is empty or missing"})

        if "cell_id" not in df.columns or "model_prediction" not in df.columns or "Capacity_Allocated(Mbps)" not in df.columns:
            return jsonify({"error": "CSV file does not have the required columns"})

        df["cell_id"] = df["cell_id"].astype(str).str.lower()
        filtered_df = df[df["cell_id"] == cell_id]
        
        if filtered_df.empty:
            return jsonify({"error": "No matching cell_id found"})
        
        latest_entry = filtered_df.sort_values(by="timestamp", ascending=False).iloc[0]
        network_traffic_usage = latest_entry.get("model_prediction", "N/A")
        resource_allocation = latest_entry.get("Capacity_Allocated(Mbps)", "N/A")
        timestamp_label = latest_entry.get("timestamp", "N/A")
        
        response_text = (f"For cell {cell_id}, the predicted network traffic usage is {network_traffic_usage}, "
                         f"and the needed resource allocation for this traffic is {resource_allocation} Mbps.")
        
        return jsonify({"response": response_text})
    except Exception as e:
        return jsonify({"error": str(e)})

# API Endpoint: Process CSV Upload
@app.route('/upload_csv', methods=['POST'])
def upload_csv():
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No file uploaded"})
        
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "No selected file"})
        
        input_df = pd.read_csv(file)
        if "cell_id" not in input_df.columns:
            return jsonify({"error": "CSV must contain a 'cell_id' column"})
        
        input_df["cell_id"] = input_df["cell_id"].astype(str).str.lower()
        df["cell_id"] = df["cell_id"].astype(str).str.lower()
        
        merged_df = input_df.merge(df, on="cell_id", how="left")
        
        # Ensure required columns exist before processing
        if "model_prediction" not in merged_df.columns:
            merged_df["model_prediction"] = "N/A"
        if "Capacity_Allocated(Mbps)" not in merged_df.columns:
            merged_df["Capacity_Allocated(Mbps)"] = "N/A"
        
        # Rename columns
        merged_df.rename(columns={
            "model_prediction": "Network Traffic Usage",
            "Capacity_Allocated(Mbps)": "Resource Allocation (Mbps)",
            "timestamp": "Last Updated"
        }, inplace=True)
        
        # Keep only necessary columns
        result_df = merged_df[["cell_id", "Network Traffic Usage", "Resource Allocation (Mbps)", "Last Updated"]]
        result_df = result_df.sort_values(by="Last Updated", ascending=False).drop_duplicates(subset=["cell_id"])
        
        output_path = "static/Predicted Cells.csv"
        result_df.to_csv(output_path, index=False)
        
        return jsonify({"download_url": f"/{output_path}"})
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.100:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Mar/2025 11:39:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 11:39:31] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Mar/2025 11:39:31] "GET /static/mtn-logo.jpg HTTP/1.1" 304 -
127.0.0.1 - - [16/Mar/2025 11:39:51] "POST /upload_csv HTTP/1.1" 200 -


In [6]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 14.5 MB/s eta 0:00:0000:01:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandasai 1.4 requires pandas==1.5.3, but you have pandas 2.2.3 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
